## Setup

In [1]:
import os, sys
sys.path.append(os.path.abspath('../../src/'))

import torch
import torch.optim as optim
import torch.nn as nn
import itertools
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR

from model.spectogram_dataset import SpectrogramDataset

import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mldogs (mldogs-wut). Use `wandb login --relogin` to force relogin


True

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


## CNN

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, dropout_conv_p=0.0, dropout_fc_p=0.0):
        super(Net, self).__init__()

        # Convolutional and pooling layers
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5)  # Output: (16, 169, 82)
        self.pool1 = nn.MaxPool2d(2, 2)              # Output: (16, 84, 41)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=5) # Output: (32, 80, 37)
        self.pool2 = nn.MaxPool2d(2, 2)              # Output: (32, 40, 18)

        # Dropout after convolutional layers
        self.dropout_conv = nn.Dropout2d(p=dropout_conv_p)  # Dropout2d for 2D data

        # Fully connected layers
        self.fc1 = nn.Linear(32 * 40 * 18, 256)      # Adjusted input size
        self.fc2 = nn.Linear(256, 128)              # Fully connected layer 2
        self.fc3 = nn.Linear(128, 1)              

        # Dropout for fully connected layers
        self.dropout_fc = nn.Dropout(p=dropout_fc_p)

    def forward(self, x):
        # Convolutional layers
        x = self.conv1(x)
        x = F.relu(x)  # Apply ReLU activation
        x = self.pool1(x)  # Pooling
        x = self.dropout_conv(x)  # Dropout (no effect if p=0.0)

        x = self.conv2(x)
        x = F.relu(x)  # Apply ReLU activation
        x = self.pool2(x)  # Pooling
        x = self.dropout_conv(x)  # Dropout (no effect if p=0.0)

        # Flatten
        x = torch.flatten(x, 1)

        # Fully connected layers
        x = self.fc1(x)
        x = F.relu(x)  # Apply ReLU activation
        x = self.dropout_fc(x)  # Dropout (no effect if p=0.0)

        x = self.fc2(x)
        x = F.relu(x)  # Apply ReLU activation
        x = self.dropout_fc(x)  # Dropout (no effect if p=0.0)
        x = torch.sigmoid(self.fc3(x))  # Output layer
        
        return x

In [4]:
from sklearn.metrics import f1_score

def train(model, criterion, optimizer, scheduler, num_epochs, train_loader, val_loader, model_name = "cnn"):
    model.to(device)
    
    PRINT_STEP = len(train_loader) // 5 - 1
    epochs_without_val_acc_improvement = 0
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        model.train()

        running_loss = 0.0
        correct = 0
        all_labels = []
        all_preds = []

        for  i, data in enumerate(train_loader):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            labels = labels.unsqueeze(1)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            preds = (outputs > 0.5).float()  # Binary prediction with threshold 0.5
            all_labels.extend(labels.cpu().numpy().flatten())
            all_preds.extend(preds.detach().cpu().numpy().flatten())

            correct += (preds == labels).float().mean().item()
            
            running_loss += loss.item()

            if i % PRINT_STEP == PRINT_STEP-1:
                accuracy = correct / PRINT_STEP
                loss = running_loss / PRINT_STEP
                step = epoch * len(train_loader) + i
                wandb.log({
                        "train/accuracy": accuracy,
                        "train/loss": loss
                    },
                    step=step
                )
                running_loss = 0.0
                correct = 0

        f1 = f1_score(all_labels, all_preds, average='macro')
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}, F1 Score: {f1:.4f}")

        if scheduler is not None:
            scheduler.step()
        
        # Validation
        model.eval()

        val_loss = 0.0
        val_correct = 0

        with torch.no_grad():
            for j, data in enumerate(val_loader):
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device)
                labels = labels.unsqueeze(1)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Binary predictions
                preds = (outputs > 0.5).float()
                val_correct += (preds == labels).float().mean().item()

        accuracy = val_correct / len(val_loader)
        loss = val_loss / len(val_loader)
        wandb.log({
                "validation/accuracy": accuracy,
                "validation/loss": loss
            },
            step=(epoch + 1) * len(train_loader)
        )

        if accuracy > best_val_acc:
            best_val_acc = accuracy
            epochs_without_val_acc_improvement = 0
        else:
            epochs_without_val_acc_improvement += 1
        if epochs_without_val_acc_improvement >= 10:
            print("8 epochs without a val accuracy improvement. Stopping the train")
            break
        if epoch > 15 and accuracy < 0.9:
            print("Too low accuracy. Stopping the train")
            break

    print("Training complete.")
    
def test(model, test_loader):
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on test images: {100 * correct // total} %')

## Data

In [5]:
from utils.dataset_creator import SpecgramsRandomFilter, SpecgramsSilentFilter

DATASETS_PARENT_PATH = 'datasets'
DATASET_PATH = 'dataset'
DATA_DIR = os.path.join('../../', DATASETS_PARENT_PATH, DATASET_PATH)

In [6]:
from prepare_datasets import create_datasets

create_datasets(DATASETS_PARENT_PATH, DATASET_PATH, [
    SpecgramsSilentFilter(1.1)
])

DatasetCreator: Exporting the dataset with the following parameters:
    n_fft=1024
    n_mels=86
    duration_seconds=2s
    multithreading=True
Class 0 recordings count: 270
Class 1 recordings count: 630
Image properties:
    width=173px
    height=86px
Removing directory /home/migoox/dev/ml-dogs-sharepoint/src/../datasets/dataset/train/0
Removing directory /home/migoox/dev/ml-dogs-sharepoint/src/../datasets/dataset/train/1
Finished [900/900]Dataset has been exported. Elapsed time: 139.80219292640686s.
DatasetCreator: Exporting the dataset with the following parameters:
    n_fft=1024
    n_mels=86
    duration_seconds=2s
    multithreading=True
Class 0 recordings count: 90
Class 1 recordings count: 210
Image properties:
    width=173px
    height=86px
Removing directory /home/migoox/dev/ml-dogs-sharepoint/src/../datasets/dataset/validation/0
Removing directory /home/migoox/dev/ml-dogs-sharepoint/src/../datasets/dataset/validation/1
Finished [300/300]Dataset has been exported. Elapse

In [8]:
INPUT_WIDTH = 67
INPUT_HEIGHT = 66

LEARNING_RATE = 0.001
LR_DECAY = 0.95
BATCH_SIZE = 32
EPOCHS = 30

DROPOUT_ARRAY = [0.2, 0.3, 0.4]


## Training

In [9]:
for d1, d2 in itertools.product(DROPOUT_ARRAY, DROPOUT_ARRAY):
    print(f"d_c:{d1}, d_f:{d2}")
    wandb.init(
        project="iml_final_tests_2",
        config={
            "learning_rate": LEARNING_RATE,
            "learning_rate_decay": LR_DECAY,
            "batch_size": BATCH_SIZE,
            "input_resolution": (INPUT_WIDTH, INPUT_HEIGHT),
            "architecture": "CNN",
            "dataset": "DAPS",
            "scheduler": "step",
            "fixed": "true"
        },
        name=f"d_c:{d1}, d_f:{d2}-sched"
    )
    model = Net(dropout_conv_p=d1, dropout_fc_p=d2)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = StepLR(optimizer, step_size=10, gamma=LR_DECAY)
    transform = transforms.ToTensor()
    
    train_dataset = SpectrogramDataset(data_dir=os.path.join(DATA_DIR, "train"), transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    val_dataset = SpectrogramDataset(data_dir=os.path.join(DATA_DIR, "validation"), transform=transform)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    test_dataset = SpectrogramDataset(data_dir=os.path.join(DATA_DIR, "test"),transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    train(model, criterion, optimizer, scheduler, EPOCHS, train_loader, val_loader, model_name="simple_cnn")

    wandb.finish()


d_c:0.2, d_f:0.2


Epoch 1/30, Loss: 0.0200, F1 Score: 0.4140
Epoch 2/30, Loss: 0.0125, F1 Score: 0.6919
Epoch 3/30, Loss: 0.0068, F1 Score: 0.8564
Epoch 4/30, Loss: 0.0059, F1 Score: 0.9044
Epoch 5/30, Loss: 0.0047, F1 Score: 0.9311
Epoch 6/30, Loss: 0.0034, F1 Score: 0.9484
Epoch 7/30, Loss: 0.0029, F1 Score: 0.9662
Epoch 8/30, Loss: 0.0025, F1 Score: 0.9708
Epoch 9/30, Loss: 0.0020, F1 Score: 0.9749
Epoch 10/30, Loss: 0.0010, F1 Score: 0.9763
Epoch 11/30, Loss: 0.0017, F1 Score: 0.9831
Epoch 12/30, Loss: 0.0017, F1 Score: 0.9813
Epoch 13/30, Loss: 0.0011, F1 Score: 0.9874
Epoch 14/30, Loss: 0.0021, F1 Score: 0.9897
Epoch 15/30, Loss: 0.0012, F1 Score: 0.9903
Epoch 16/30, Loss: 0.0004, F1 Score: 0.9905
Epoch 17/30, Loss: 0.0003, F1 Score: 0.9910
Epoch 18/30, Loss: 0.0005, F1 Score: 0.9918
Epoch 19/30, Loss: 0.0006, F1 Score: 0.9928
Epoch 20/30, Loss: 0.0003, F1 Score: 0.9914
Epoch 21/30, Loss: 0.0003, F1 Score: 0.9946
Epoch 22/30, Loss: 0.0003, F1 Score: 0.9936
Epoch 23/30, Loss: 0.0010, F1 Score: 0.99

train/accuracy,▁▁▁▁▂▄▄▅▆▇▇▇▇▇▇█████████████████████████
train/loss,██▇▅▃▃▃▂▂▂▂▂▂▂▂▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation/accuracy,▁▄▅▆▆▆▇▇▇▇▇▇██▇██▇███▇████████
validation/loss,█▆▄▄▃▃▃▄▄▃▂▁▁▂▅▂▂▃▃▃▄▅▃▃▄▄▂▃▃▂
train/accuracy,0.99821
train/loss,0.00533
validation/accuracy,0.94937
validation/loss,0.274


d_c:0.2, d_f:0.3


Epoch 1/30, Loss: 0.0211, F1 Score: 0.4315
Epoch 2/30, Loss: 0.0182, F1 Score: 0.3996
Epoch 3/30, Loss: 0.0132, F1 Score: 0.5732
Epoch 4/30, Loss: 0.0081, F1 Score: 0.8449
Epoch 5/30, Loss: 0.0060, F1 Score: 0.9070
Epoch 6/30, Loss: 0.0039, F1 Score: 0.9363
Epoch 7/30, Loss: 0.0041, F1 Score: 0.9541
Epoch 8/30, Loss: 0.0022, F1 Score: 0.9673
Epoch 9/30, Loss: 0.0016, F1 Score: 0.9714
Epoch 10/30, Loss: 0.0016, F1 Score: 0.9737
Epoch 11/30, Loss: 0.0022, F1 Score: 0.9795
Epoch 12/30, Loss: 0.0012, F1 Score: 0.9815
Epoch 13/30, Loss: 0.0008, F1 Score: 0.9838
Epoch 14/30, Loss: 0.0012, F1 Score: 0.9883
Epoch 15/30, Loss: 0.0016, F1 Score: 0.9893
Epoch 16/30, Loss: 0.0012, F1 Score: 0.9907
Epoch 17/30, Loss: 0.0012, F1 Score: 0.9909
Epoch 18/30, Loss: 0.0009, F1 Score: 0.9883
Epoch 19/30, Loss: 0.0004, F1 Score: 0.9911
Epoch 20/30, Loss: 0.0001, F1 Score: 0.9920
Epoch 21/30, Loss: 0.0010, F1 Score: 0.9916
Epoch 22/30, Loss: 0.0002, F1 Score: 0.9942
Epoch 23/30, Loss: 0.0016, F1 Score: 0.99

...

Epoch 1/30, Loss: 0.0199, F1 Score: 0.4113
Epoch 2/30, Loss: 0.0173, F1 Score: 0.5050
Epoch 3/30, Loss: 0.0116, F1 Score: 0.7846
Epoch 4/30, Loss: 0.0066, F1 Score: 0.8759
Epoch 5/30, Loss: 0.0053, F1 Score: 0.9220
Epoch 6/30, Loss: 0.0039, F1 Score: 0.9431
Epoch 7/30, Loss: 0.0036, F1 Score: 0.9546
Epoch 8/30, Loss: 0.0045, F1 Score: 0.9709
Epoch 9/30, Loss: 0.0024, F1 Score: 0.9723
Epoch 10/30, Loss: 0.0009, F1 Score: 0.9780
Epoch 11/30, Loss: 0.0023, F1 Score: 0.9803
Epoch 12/30, Loss: 0.0013, F1 Score: 0.9807
Epoch 13/30, Loss: 0.0008, F1 Score: 0.9844
Epoch 14/30, Loss: 0.0019, F1 Score: 0.9872
Epoch 15/30, Loss: 0.0012, F1 Score: 0.9875
Epoch 16/30, Loss: 0.0007, F1 Score: 0.9866
Epoch 17/30, Loss: 0.0005, F1 Score: 0.9881
Epoch 18/30, Loss: 0.0026, F1 Score: 0.9897
Epoch 19/30, Loss: 0.0017, F1 Score: 0.9858
Epoch 20/30, Loss: 0.0015, F1 Score: 0.9875
Epoch 21/30, Loss: 0.0005, F1 Score: 0.9870
Epoch 22/30, Loss: 0.0014, F1 Score: 0.9875
Epoch 23/30, Loss: 0.0003, F1 Score: 0.99

train/accuracy,▁▁▁▃▄▇▆▇▇▇▇▇▇███████████████████████████
train/loss,███▇▇▅▄▃▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation/accuracy,▁▃▅▆▆▇▇▇▇▇▇▇█▇████████████████
validation/loss,█▇▆▅▃▂▁▃▂▂▃▂▃▄▂▂▃▄▂▁▁▂▄▂▂▂▂▂▁▁
train/accuracy,0.99554
train/loss,0.01084
validation/accuracy,0.96169
validation/loss,0.19066


d_c:0.4, d_f:0.4


Epoch 1/30, Loss: 0.0195, F1 Score: 0.4008
Epoch 2/30, Loss: 0.0185, F1 Score: 0.4000
Epoch 3/30, Loss: 0.0150, F1 Score: 0.5589
Epoch 4/30, Loss: 0.0134, F1 Score: 0.7668
Epoch 5/30, Loss: 0.0123, F1 Score: 0.8361
Epoch 6/30, Loss: 0.0083, F1 Score: 0.8836
Epoch 7/30, Loss: 0.0077, F1 Score: 0.8962
Epoch 8/30, Loss: 0.0053, F1 Score: 0.9190
Epoch 9/30, Loss: 0.0036, F1 Score: 0.9361
Epoch 10/30, Loss: 0.0054, F1 Score: 0.9388
Epoch 11/30, Loss: 0.0029, F1 Score: 0.9448
Epoch 12/30, Loss: 0.0028, F1 Score: 0.9551
Epoch 13/30, Loss: 0.0020, F1 Score: 0.9600
Epoch 14/30, Loss: 0.0053, F1 Score: 0.9671
Epoch 15/30, Loss: 0.0015, F1 Score: 0.9672
Epoch 16/30, Loss: 0.0029, F1 Score: 0.9754
Epoch 17/30, Loss: 0.0026, F1 Score: 0.9774
Epoch 18/30, Loss: 0.0017, F1 Score: 0.9807
Epoch 19/30, Loss: 0.0012, F1 Score: 0.9811
Epoch 20/30, Loss: 0.0009, F1 Score: 0.9817
Epoch 21/30, Loss: 0.0020, F1 Score: 0.9852
Epoch 22/30, Loss: 0.0015, F1 Score: 0.9839
Epoch 23/30, Loss: 0.0008, F1 Score: 0.99

train/accuracy,▁▁▁▁▂▄▅▅▅▆▆▆▇▇▇▇████████████████████████
train/loss,██▇▇▇▆▅▅▅▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation/accuracy,▁▁▃▄▅▆▆▆▇▆▇▇▇▇█▇▇█▇▇█▇▇███████
validation/loss,▇█▇▇▆▄▃▃▃▄▂▄▃▂▁▃▂▂▃▃▂▂▃▂▂▁▁▁▂▁
train/accuracy,0.99107
train/loss,0.02422
validation/accuracy,0.96787
validation/loss,0.14583


In [19]:
wandb.finish()